# 1. 实战Kaggle比赛狗的品种识别ImageNetDogs

① 比赛网址是 https://www.kaggle.com/c/dog-breed-identification

In [2]:
import os  # [语法]: 导入标准库 os (Operating System)。 [作用]: 提供与操作系统交互的功能，比如路径拼接、创建文件夹、列出文件名。
import shutil  # [语法]: 导入标准库 shutil (Shell Utilities)。 [作用]: 提供高级文件操作，最主要就是用来复制(copy)和移动文件。
import pandas as pd  # [语法]: 导入 pandas 库并起别名 pd。 [作用]: 数据分析的神器，这里专门用来读取和处理 .csv 格式的标签文件。
from tqdm import tqdm  # [语法]: 从 tqdm 库导入 tqdm 模块。 [作用]: 显示进度条。在处理成千上万张图片时，能让你知道代码跑到哪了，不至于干等。

# 1. 定义路径 (根据你的实际存放位置修改)
# [语法]: 变量赋值 (字符串)。
# [作用]: 定义所有数据存放的根目录。所有的子文件夹都会在这个目录下。
DATA_ROOT = './kaggle_dog'

# [语法]: os.path.join 路径拼接。
# [作用]: 拼接出原始训练集图片的存放路径 (例如 ./kaggle_dog/train)。它会自动适配 Windows(\) 和 Linux(/) 的分隔符。
ORIGIN_TRAIN_DIR = os.path.join(DATA_ROOT, 'train')

# [语法]: os.path.join 路径拼接。
# [作用]: 拼接出原始测试集图片的存放路径。
ORIGIN_TEST_DIR = os.path.join(DATA_ROOT, 'test')

# [语法]: os.path.join 路径拼接。
# [作用]: 拼接出标签文件 labels.csv 的完整路径。
LABEL_CSV = os.path.join(DATA_ROOT, 'labels.csv')

# 目标路径: 我们把整理好的图片放到这里
# [语法]: os.path.join 路径拼接。
# [作用]: 定义一个新文件夹的路径，准备用来存放整理后(按品种分类)的训练图片。
TARGET_TRAIN_DIR = os.path.join(DATA_ROOT, 'train_ready')

# [语法]: os.path.join 路径拼接 (多层)。
# [作用]: 定义整理后的测试集路径。
# ★关键点★：为什么要加 'unknown'？因为 PyTorch 的 ImageFolder 读取数据时，强制要求格式为 root/类别名/图片。
# 测试集没有类别，所以我们需要人为制造一个伪类别文件夹(unknown)，否则代码会报错。
TARGET_TEST_DIR = os.path.join(DATA_ROOT, 'test_ready', 'unknown') 

# [语法]: 定义函数 organize_data。
# [作用]: 封装数据整理的所有逻辑，方便一键调用。
def organize_data():
    
    # [语法]: os.path.exists 判断路径是否存在。
    # [作用]: 这是一个“防呆设计”。如果代码以前跑过，文件夹已经有了，就直接跳过，避免重复复制浪费时间。
    if os.path.exists(TARGET_TRAIN_DIR):
        
        # [语法]: 打印提示信息。
        # [作用]: 告诉用户程序为什么没有干活。
        print("数据好像已经整理好了，跳过此步。")
        
        # [语法]: return 语句。
        # [作用]: 结束当前函数的执行，直接跳出。
        return

    # [语法]: 打印提示信息。
    # [作用]: 提示用户程序开始处理训练集了。
    print("开始整理训练集数据 (Copied)...")
    
    # [语法]: pd.read_csv 读取 CSV 文件。
    # [作用]: 将 labels.csv 读取为一个 DataFrame (表格对象)，方便后续查每一张图对应的品种。
    df = pd.read_csv(LABEL_CSV)
    
    # [语法]: for 循环 + tqdm 进度条 + df.iterrows() 迭代器。
    # [作用]: 
    # 1. df.iterrows(): 把表格一行一行地取出来。
    # 2. tqdm(...): 给这个循环加个进度条，显示 "处理到第几行了 / 总共多少行"。
    # 3. total=len(df): 告诉进度条总数，方便计算百分比。
    for idx, row in tqdm(df.iterrows()):
        
        # [语法]: 字典/Series 取值。
        # [作用]: 获取当前这一行里的 'id' 列的值 (即图片文件名，不含后缀)。
        img_id = row['id']
        
        # [语法]: 字典/Series 取值。
        # [作用]: 获取当前这一行里的 'breed' 列的值 (即狗的品种，也就是我们要创建的文件夹名)。
        breed = row['breed']
        
        # [语法]: os.path.join + f-string 格式化字符串。
        # [作用]: 拼凑出这张图片现在的完整路径。例如: ./kaggle_dog/train/001.jpg。
        src = os.path.join(ORIGIN_TRAIN_DIR, f"{img_id}.jpg")
        
        # [语法]: os.path.join 路径拼接。
        # [作用]: 拼凑出这张图片想去的地方。例如: ./kaggle_dog/train_ready/husky。
        dst_dir = os.path.join(TARGET_TRAIN_DIR, breed)
        
        # [语法]: if 判断 + not 取反。
        # [作用]: 检查目标品种文件夹是否存在。如果不存在(not exists)，才需要创建。
        if not os.path.exists(dst_dir):
            
            # [语法]: os.makedirs 创建目录。
            # [作用]: 创建对应的品种文件夹。makedirs 支持递归创建(如果上级目录也没有会一起创建)。
            os.makedirs(dst_dir)
            
        # [语法]: shutil.copy 文件复制。
        # [作用]: 核心动作！把图片从源路径(src) 复制到 目标文件夹下的同名文件。
        # f"{img_id}.jpg" 是为了保持文件名不变。
        shutil.copy(src, os.path.join(dst_dir, f"{img_id}.jpg"))

    # [语法]: 打印提示信息。
    # [作用]: 训练集搞定，提示开始搞测试集。
    print("开始整理测试集数据...")
    
    # [语法]: if 判断路径是否存在。
    # [作用]: 检查测试集的目标文件夹 ./kaggle_dog/test_ready/unknown 是否存在。
    if not os.path.exists(TARGET_TEST_DIR):
        
        # [语法]: os.makedirs 创建目录。
        # [作用]: 如果不存在，就创建这个目录。
        os.makedirs(TARGET_TEST_DIR)
        
    # [语法]: for 循环 + os.listdir 列出文件。
    # [作用]: 遍历原始测试集文件夹(ORIGIN_TEST_DIR)下的所有文件名。
    # tqdm 用来显示进度。
    for img_name in tqdm(os.listdir(ORIGIN_TEST_DIR)):
        
        # [语法]: os.path.join 路径拼接。
        # [作用]: 确定这张测试图现在的绝对路径。
        src = os.path.join(ORIGIN_TEST_DIR, img_name)
        
        # [语法]: os.path.join 路径拼接。
        # [作用]: 确定这张测试图要复制去的地方 (统一放到 unknown 文件夹下)。
        dst = os.path.join(TARGET_TEST_DIR, img_name)
        
        # [语法]: shutil.copy 文件复制。
        # [作用]: 执行复制操作。
        shutil.copy(src, dst)
        
    # [语法]: 打印提示信息。
    # [作用]: 告诉用户所有工作全部完成，可以进行下一步训练了。
    print("所有数据整理完毕！")

# [语法]: 函数调用。
# [作用]: 真正执行上面定义好的 organize_data 函数，开始干活。
organize_data()

RuntimeError: CPU dispatcher tracer already initlized

开始整理训练集数据 (Copied)...


10222it [00:25, 400.90it/s]


开始整理测试集数据...


100%|██████████| 10357/10357 [00:29<00:00, 350.62it/s]

所有数据整理完毕！


In [ ]:
import torchvision.transforms as transforms  
# [语法]: 导入 torchvision.transforms 模块，并简写为 transforms。
# [作用]: 这是一个图像处理工具箱，提供了裁剪、翻转、颜色调整、归一化等一系列功能。

# ===========================
# 1. 定义训练集预处理策略
# ===========================
# [语法]: 变量赋值。
# [作用]: 训练集的数据增强策略。我们要故意把图片变得“难认”一点，强迫模型学到真正的特征，而不是死记硬背。
train_transform = transforms.Compose([
    
    # [语法]: 随机缩放裁剪 (RandomResizedCrop)。
    # 参数 224: 最终裁剪出的图片大小必须是 224x224 (这是 ResNet 模型的标准入口尺寸)。
    # scale=(0.08, 1.0): 随机选取原图 8% 到 100% 的区域。
    # ratio=(3.0/4.0, 4.0/3.0): 随机选取长宽比在 3/4 到 4/3 之间的区域。
    # [作用]: 
    # 这是最强的增强手段！模拟物体可能出现在图片的任何位置、任何大小（比如狗有时候在角落，有时候占满全屏）。
    # 它强迫模型去识别“狗的局部”也能认出是狗。
    transforms.RandomResizedCrop(224, scale=(0.08, 1.0), ratio=(3.0/4.0, 4.0/3.0)),
    
    # [语法]: 随机水平翻转 (RandomHorizontalFlip)。默认概率 p=0.5。
    # [作用]: 
    # 模拟“照镜子”。因为一只狗头朝左是狗，头朝右还是狗。
    # 这能直接让训练数据量“翻倍”，防止模型只认识头朝左的狗。
    transforms.RandomHorizontalFlip(),
    
    # [语法]: 颜色抖动 (ColorJitter)。
    # brightness=0.4: 亮度随机变化 ±40%。
    # contrast=0.4: 对比度随机变化 ±40%。
    # saturation=0.4: 饱和度随机变化 ±40%。
    # [作用]: 
    # 模拟不同的光照环境（比如白天、阴天、黄昏）。
    # 防止模型根据背景颜色来作弊（比如误以为“草地上的都是狗，沙发上的都是猫”）。
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    
    # [语法]: 转为 Tensor (ToTensor)。
    # [作用]: 
    # 1. 格式转换：把 PIL 图片 (整数 0-255, HxWxC) 变成 PyTorch Tensor (浮点数 0.0-1.0, CxHxW)。
    # 2. 维度重排：PyTorch 卷积层要求“通道数”在前，这一步自动处理了。
    # ★注意★：这一步必须在 Normalize 之前。
    transforms.ToTensor(),
    
    # [语法]: 标准化 (Normalize)。
    # 第一个列表是 RGB 三个通道的均值 (Mean)，第二个列表是标准差 (Std)。
    # [作用]: 
    # 公式：result = (input - mean) / std。
    # 为什么要用这几个奇怪的数字？因为这是 ImageNet 数据集百万张图片的统计结果。
    # 既然我们要用 ImageNet 预训练好的 ResNet，就必须把我们的图片分布拉到和 ImageNet 一样，模型才能正常工作。
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# ===========================
# 2. 定义测试/验证集预处理策略
# ===========================
# [语法]: 变量赋值。
# [作用]: 测试集不需要“花里胡哨”的随机变换，我们需要一个稳定、固定的输入，来公平地测试模型到底准不准。
val_test_transform = transforms.Compose([
    
    # [语法]: 调整大小 (Resize)。
    # 参数 256: 把图片短边缩放到 256 像素，长边按比例缩放。
    # [作用]: 
    # 先把图变到 256 大小，比需要的 224 稍微大一点点，为下一步“中心裁剪”留出余地。
    # (如果不先 Resize 直接 Crop，可能会切掉太多东西)。
    transforms.Resize(256),
    
    # [语法]: 中心裁剪 (CenterCrop)。
    # 参数 224: 从图片的正中心，切出一块 224x224 的区域。
    # [作用]: 
    # 这是测试时的标准操作。我们默认物体大概率在图片中间。
    # 不使用 RandomCrop，因为测试必须是可复现的（每次跑结果都得一样）。
    transforms.CenterCrop(224),
    
    # [语法]: 转为 Tensor。
    # [作用]: 同上，转成模型能读懂的 0-1 之间的浮点张量。
    transforms.ToTensor(),
    
    # [语法]: 标准化。
    # [作用]: 同上，使用和训练集完全一样的参数进行归一化。
    # ★关键点★：测试集必须用和训练集一样的均值/标准差，千万不能重新算。
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
import torch
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import ImageFolder
import os

# 1. 准备两个“源”数据集 (Source Datasets)
# 注意：这步很快，因为 ImageFolder 只读文件名，不读图片，所以内存开销很小。
# Source A: 专门用于训练，带有数据增强 (train_transform) imagefolder会自动加标签并且从0开始，并且字典序从小到大
train_source = ImageFolder(os.path.join(DATA_ROOT, 'train_ready'), transform=train_transform)

# Source B: 专门用于验证，不带增强 (val_test_transform)
val_source = ImageFolder(os.path.join(DATA_ROOT, 'train_ready'), transform=val_test_transform)

# 2. 手动生成切分索引 (Indices)
# 获取数据总数
num_data = len(train_source)
# 生成一个乱序的索引列表，比如 [3, 9, 1, 5, ...]
indices = torch.randperm(num_data).tolist()  #生成全排列

# 计算切分点 (90% 处)
split_point = int(0.9 * num_data)

# 切蛋糕：前 90% 给训练，后 10% 给验证
train_indices = indices[:split_point]
val_indices = indices[split_point:]

# 3. 创建最终的 Dataset (Subset)
# [语法]: Subset(源数据集, 索引列表)
# [作用]: 
# train_ds 只会去 train_source 里取 train_indices 指定的那些图 (带增强)。
# val_ds 只会去 val_source 里取 val_indices 指定的那些图 (不带增强)。
train_ds = Subset(train_source, train_indices)
val_ds = Subset(val_source, val_indices)

# 4. 放入 DataLoader (和之前一样)
train_iter = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=4)
val_iter = DataLoader(val_ds, batch_size=64, shuffle=False, num_workers=4)

# 5. 测试集 (和之前一样)
test_ds = ImageFolder(os.path.join(DATA_ROOT, 'test_ready'), transform=val_test_transform)
test_iter = DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=4)

print(f"训练集: {len(train_ds)} (使用增强变换)")
print(f"验证集: {len(val_ds)} (使用普通变换)")

训练集: 9199 (使用增强变换)
验证集: 1023 (使用普通变换)


In [5]:
from torch import nn
import torchvision
class get_net(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = torchvision.models.resnet34(pretrained=True)
        for param in self.features.parameters():
        # [语法]: 设置 requires_grad = False
        # [作用]: 
        # 告诉 PyTorch 的自动求导引擎：“在反向传播时，不要计算这些参数的梯度！”
        # 结果就是：优化器（SGD/Adam）更新参数时，这部分的权重完全不会变。
        # 既节省了显存，又加快了训练速度，还保留了大神的智慧。
            param.requires_grad = False
        self.output_new = nn.Sequential(
            nn.Linear(1000, 256),
            nn.ReLU(),
            nn.Linear(256, 120)
        )
    def forward(self,x):
        x = self.features(x)
        x = self.output_new(x)
        return x


In [9]:
# [语法]: 定义函数 evaluate_loss
# [参数]: 
# - data_iter: 数据加载器 (DataLoader)，里面装着验证集或测试集的数据。
# - net: 训练好的模型。
# - devices: 设备列表 (如 [cuda:0])。
loss = nn.CrossEntropyLoss(reduction= 'none')
def evaluate_loss(data_iter, net, devices):
    
    # [语法]: 多变量初始化
    # [作用]: 
    # l_sum: 用来累加所有图片的“总扣分” (Total Loss)。
    # n: 用来累加一共看了多少张图片 (Total Samples)。
    l_sum, n = 0.0, 0
    
    # [语法]: 遍历 DataLoader
    # [作用]: 一批一批地从数据集中取出图片 (features) 和标签 (labels)。
    for features, labels in data_iter:
        
        # [语法]: Tensor.to(device)
        # [作用]: 把数据搬运到 GPU 上 (devices[0])，因为模型在 GPU 上，数据也必须过去才能计算。
        features, labels = features.to(devices), labels.to(devices)
        
        # [语法]: 模型前向传播 (Forward)
        # [作用]: 让模型看图猜答案，得到预测结果 outputs。
        outputs = net(features)
        
        # [语法]: 计算损失
        # [作用]: 对比预测值 outputs 和真实值 labels。
        # ★关键★: 因为上面设了 reduction='none'，这里返回的 l 是一个向量 (Vector)，
        # 比如 batch_size=64，l 的形状就是 [64]，代表这 64 张图各自的 loss。
        l = loss(outputs, labels)
        
        # [语法]: l.sum() 求和
        # [作用]: 把这一个 batch 里所有图片的 loss 加起来，累加到总 loss (l_sum) 里。
        l_sum += l.sum().item()
        
        # [语法]: labels.numel() 获取元素个数 (Number of Elements)
        # [作用]: 统计这个 batch 里一共有多少张图 (比如 64 张)，累加到总数 n 里。
        n += labels.shape[0]
    
    # [语法]: 除法运算
    # [作用]: 总 Loss / 总图片数 = 平均 Loss。
    # 这就是我们要的最终指标：平均每一张图猜错的程度。
    return l_sum / n

In [ ]:
import time  # [语法]: 导入标准时间库。 [作用]: 用于记录训练耗时，计算每秒能处理多少张图片。
import torch # [语法]: 导入 PyTorch 核心库。 [作用]: 深度学习的基础框架。
from torch import nn # [语法]: 从 PyTorch 导入神经网络模块。 [作用]: 包含各种层（Linear, Conv2d）和容器（Sequential）。
from cutmix and mixup 模版.utils_aug import cutmix_data, mixup_data, MixCriterion
# [语法]: 定义训练函数 train。
# [作用]: 封装整个训练流程，包括前向传播、反向传播、参数更新和验证。
def train(net, train_iter, valid_iter, num_epochs, lr, wd, devices, lr_period, lr_decay):
    
    # [语法]: 初始化 SGD (随机梯度下降) 优化器。
    # [作用]: 它是“教”模型学习的老师。
    # 1. [param for ... if ...]: 这是一个列表推导式。它只把那些 requires_grad=True (没被冻结) 的参数传给优化器。
    #    (微调时，ResNet主体的参数被冻结了，不需要更新，所以不能传进去，否则报错或浪费资源)。
    # 2. lr=lr: 学习率，决定每次参数更新的步子迈多大。
    # 3. momentum=0.9: 动量。让优化器在下坡时有“惯性”，不容易卡在局部坑里，收敛更快。
    # 4. weight_decay=wd: L2 正则化。惩罚过大的权重值，防止模型死记硬背 (过拟合)。
    trainer = torch.optim.SGD(
        [param for param in net.parameters() if param.requires_grad],
        lr=lr, momentum=0.9, weight_decay=wd)
    
    # [语法]: 初始化学习率调度器 (Scheduler)。
    # [作用]: 实现“学习率衰减”策略。
    # StepLR: 每过 lr_period 个 Epoch，就把学习率乘以 lr_decay。
    # 例如: 初始lr=0.01, period=10, decay=0.1 -> 第10轮时 lr 变成 0.001。
    # 目的: 训练后期步子要迈小点，才能精细地找到最优解。
    scheduler = torch.optim.lr_scheduler.StepLR(trainer, lr_period, lr_decay)

    # [语法]: for 循环 (Epoch Loop)。
    # [作用]: 开始多轮训练。range(num_epochs) 生成从 0 到 num_epochs-1 的序列。
    for epoch in range(num_epochs):
        
        # [语法]: 开启训练模式。
        # [作用]: 这一步**至关重要**！
        # 它会通知模型里的 Dropout 层开始随机丢弃神经元，通知 BatchNorm 层开始根据当前 Batch 更新均值和方差。
        # 如果不写这句，模型可能无法正常训练。
        net.train()
        
        # [语法]: 多变量赋值初始化。
        # [作用]: 初始化本轮 (Epoch) 的统计指标。
        # train_l_sum: 累加这一轮所有样本的总损失值。
        # train_n: 累加这一轮一共训练了多少张图片。
        train_l_sum, train_n = 0.0, 0
        
        # [语法]: time.time() 获取当前时间戳。
        # [作用]: 记录这一轮开始的时间点，用于后面计算这一轮跑了多久。
        start_time = time.time()
        
        # [语法]: for 循环 + enumerate (Batch Loop)。
        # [作用]: 从 DataLoader 中一批一批地取数据。
        # i 是当前的批次索引 (0, 1, 2...)，(features, labels) 是图片和标签。
        for i, (features, labels) in enumerate(train_iter):
            
            # [语法]: Tensor.to(device) 数据迁移。
            # [作用]: 把 CPU 上的图片和标签搬运到 GPU 上 (devices[0])，必须和模型在同一个设备上才能计算。
            features, labels = features.to(devices), labels.to(devices)
            
            # [语法]: 优化器梯度清零。
            # [作用]: 清空上一个 Batch 残留的梯度信息。
            # PyTorch 默认会累加梯度，如果不清零，梯度的方向就全乱了，模型会学废。
            trainer.zero_grad()
            
            # [语法]: 模型前向传播 (Forward Pass)。
            # [作用]: 把图片喂给模型，计算出预测结果 output。
            output = net(features)
            
            # [语法]: 计算损失 (Calculate Loss)。
            # [作用]: 
            # 1. loss(output, labels): 调用之前定义的 CrossEntropyLoss (reduction='none')，算出一批 loss 向量。
            # 2. .sum(): 把这个 Batch 里所有图片的 loss 加起来，变成一个标量 (Scalar)。
            # *注意*: 这里加起来是为了后面方便做反向传播。
            l = loss(output, labels).sum()
            
            # [语法]: 反向传播 (Backward Pass)。
            # [作用]: 根据损失 l，利用链式法则自动计算出每个参数的梯度 (Gradients)。
            l.backward()
            
            # [语法]: 参数更新 (Optimizer Step)。
            # [作用]: 老师根据刚才算出的梯度，修正模型的参数权重。
            trainer.step()
            
            # [语法]: 累加统计。
            # [作用]: 
            # 1. l.item(): 把 Tensor 里的数值取出来变成 Python float。
            # 2. labels.shape[0]: 获取当前 Batch 有多少张图 (比如 64 张)。
            train_l_sum += l.item()
            train_n += labels.shape[0]
        
        # [语法]: 调度器更新 (Scheduler Step)。
        # [作用]: 这一轮 Epoch 跑完了，告诉调度器一声。
        # 调度器会检查“现在是第几轮了？”，如果到了 lr_period 的倍数，它就会把学习率调小。
        scheduler.step()
        
        # [语法]: if 判断语句。
        # [作用]: 检查是否提供了验证集 (valid_iter)。如果是 None，说明不需要验证。
        if valid_iter is not None:
            
            # [语法]: 调用自定义函数 evaluate_loss。
            # [作用]: 让模型在验证集上跑一遍，看看它在没见过的题上能考多少分。
            valid_loss = evaluate_loss(valid_iter, net, devices)
        
        # [语法]: 数学运算 (计算耗时)。
        # [作用]: 当前时间 - 开始时间 = 这一轮花了几秒。
        epoch_duration = time.time() - start_time
        
        # [语法]: 数学运算 (计算吞吐量)。
        # [作用]: 这一轮处理的总图数 / 耗时 = 每秒能处理多少张图 (samples/sec)。
        # 这是衡量显卡性能的重要指标。
        throughput = train_n / epoch_duration
        
        # [语法]: f-string 字符串格式化。
        # [作用]: 构造要打印的日志信息。
        # :.3f 表示保留小数点后 3 位。
        # train_l_sum / train_n = 平均训练损失。
        print(f'Epoch {epoch + 1}/{num_epochs}: ')
        print(f'Train Loss: {train_l_sum / train_n:.3f}')
        
        # [语法]: if 判断。
        # [作用]: 如果有验证集，就把验证集损失拼接到日志字符串后面。
        if valid_iter is not None:
            print(f'Value Loss: {valid_loss}')
        # [语法]: 打印输出。
        # [作用]: 单独打印速度信息，方便监控硬件性能。
        print(f'   Speed: {throughput:.1f} samples/sec on {str(devices)}')

In [15]:
import torch
loss = nn.CrossEntropyLoss(reduction='none')
# ===========================
# 1. 自动检测硬件设备 (替换 d2l.try_all_gpus)
# ===========================
# [语法]: 定义 get_all_devices 函数
# [作用]: 模仿 d2l 的功能。
# 1. torch.cuda.device_count(): 检查有几张显卡。
# 2. 列表推导式: 如果有卡，生成 [cuda:0, cuda:1...] 的列表。
# 3. else: 如果没卡，返回 [cpu]。
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ===========================
# 2. 设置超参数 (Hyperparameters)
# ===========================
# [语法]: 多变量赋值
# [作用]: 
# num_epochs = 10: 训练 10 轮。
# lr = 1e-4: 学习率 0.0001 (微调通常用很小的学习率，防止把预训练好的权重改坏了)。
# wd = 1e-4: 权重衰减 (Weight Decay)，防止过拟合。
num_epochs, lr, wd = 10, 1e-4, 1e-4

# [语法]: 学习率衰减参数
# [作用]: 
# lr_period = 2: 每隔 2 轮调整一次学习率。
# lr_decay = 0.9: 每次调整为原来的 90% (即乘以 0.9)。
lr_period, lr_decay = 2, 0.9

# ===========================
# 3. 初始化模型
# ===========================
# [语法]: 调用之前定义的 get_net 函数
# [作用]: 
# 1. 下载 ResNet34。
# 2. 冻结前层参数。
# 3. 替换最后一层为 120 分类。
# 4. 把模型搬到 GPU 上。
net = get_net()
net = net.to(device)

# ===========================
# 4. 开始训练
# ===========================
# [语法]: 调用训练函数
# [作用]: 
# 把模型(net)、数据(iter)、超参全部传进去，开始跑代码。
# 这一步会打印出每一轮的 Loss 和 速度。
train(net, train_iter, val_iter, num_epochs, lr, wd,  device , lr_period, lr_decay)

Epoch 1/10: 
Train Loss: 2.585
Value Loss: 0.994572864133475
   Speed: 204.3 samples/sec on cuda
Epoch 2/10: 
Train Loss: 1.431
Value Loss: 0.8130415863305592
   Speed: 215.8 samples/sec on cuda
Epoch 3/10: 
Train Loss: 1.292
Value Loss: 0.74445712927616
   Speed: 209.9 samples/sec on cuda
Epoch 4/10: 
Train Loss: 1.212
Value Loss: 0.736484542503618
   Speed: 180.2 samples/sec on cuda
Epoch 5/10: 
Train Loss: 1.140
Value Loss: 0.7128385840273322
   Speed: 176.6 samples/sec on cuda
Epoch 6/10: 
Train Loss: 1.108
Value Loss: 0.7039528154091634
   Speed: 190.8 samples/sec on cuda
Epoch 7/10: 
Train Loss: 1.074
Value Loss: 0.6832323568424288
   Speed: 214.5 samples/sec on cuda
Epoch 8/10: 
Train Loss: 1.039
Value Loss: 0.6759233726434344
   Speed: 198.5 samples/sec on cuda
Epoch 9/10: 
Train Loss: 1.040
Value Loss: 0.6713126733505831
   Speed: 203.4 samples/sec on cuda
Epoch 10/10: 
Train Loss: 1.031
Value Loss: 0.6400620114651826
   Speed: 212.8 samples/sec on cuda


In [21]:
import os
import torch
import torch.nn.functional as F # [语法]: 通常我们会把 functional 简写为 F

# [语法]: 初始化列表
# [作用]: 用来存放所有预测结果（概率值）。
preds = []


with torch.no_grad(): 
# [语法]: 遍历测试集 DataLoader
# [作用]: 这里的 test_iter 应该是之前定义的DataLoader，batch_size=64。
# 注意: test_iter 里虽然有 label，但在 Kaggle 测试集中 label 通常是占位符（或者空），我们不用它。
    for data, label in test_iter:
        
        # [语法]: 模型推理 + Softmax 归一化
        # [作用]: 
        # 1. net(data.to(...)): 把图片搬到 GPU 算出预测分数 (Logits)。此时形状是 [64, 120]。
        # 2. F.softmax(..., dim=1): 核心！把分数变成概率 (0-1之间，和为1)。
        # 必须改为 dim=1 (按行/类别归一化)。
        # 含义：对于“每一张图”，它属于120个类别的概率之和应该是1。
        data = data.to(device)
        output = F.softmax(net(data), dim=1) 
        
        #extend 和 append 的区别是 , append 是 直接 加到末尾 ， extend 是 把传入内容一个个拆掉，然后加入末尾
        preds.extend(output.tolist())

        # [语法]: 打印进度
        # [作用]: 看着不断增加的数字 (64, 128, 192...)，知道程序在动。
        print(len(preds))

# [语法]: 获取文件名列表 (IDs)
# [作用]: 
# 1. os.listdir(...): 读取测试集文件夹下的所有图片文件名 (如 '00a3edd22.jpg')。
# 2. sorted(): ⚠️非常重要！ImageFolder 读取数据时是按文件名排序的。
# 我们生成的 preds 列表也是按这个顺序排列的。
# 所以这里获取文件名时，必须也 sort 一下，确保“文件名”和“预测结果”是一一对应的。
# *注*: 这里的路径要改成你之前定义的 TARGET_TEST_DIR (比如 'kaggle_dog/test_ready/unknown')。
ids = sorted(os.listdir(os.path.join(DATA_ROOT, 'test_ready', 'unknown')))    

# [语法]: open 函数 + with 上下文管理器
# [参数]: 'w' 表示 write (写入模式)。如果文件不存在会新建，如果存在会清空重写。
# [作用]: 
# 1. 安全打开文件。
# 2. 无论中间代码是否报错，with 语句结束时都会自动关闭文件 (f.close())，防止数据丢失。
with open('submission.csv', 'w') as f:
    
    # ===========================
    # 1. 写入表头 (Header)
    # ===========================
    # [语法]: 字符串拼接 ('id,' + ...) 和 join 方法。
    # [作用]: 
    # Kaggle 规定第一行必须是列名。格式: id, breed1, breed2, ...
    # train_source.classes: 这是一个列表 ['affenpinscher', 'beagle', ...]。
    # ','.join(...): 把列表里的字符串用逗号连成一长串。
    # '\n': 换行符，写完表头必须换行，否则数据会接到表头后面。
    f.write('id,' + ','.join(train_source.classes) + '\n')
    
    # ===========================
    # 2. 遍历数据 (Loop)
    # ===========================
    # [语法]: zip(列表A, 列表B) 打包函数。
    # [作用]: 
    # ids 是文件名列表 ['001.jpg', '002.jpg']。
    # preds 是预测概率列表 [[0.1, 0.9], [0.8, 0.2]]。
    # zip 会把它们“拉链”一样一对一锁死，每次循环吐出一组 (i=文件名, output=概率向量)。
    for i, output in zip(ids, preds):
        
        # ===========================
        # 3. 写入每一行数据 (Data Row)
        # ===========================
        # 这行代码信息量很大，我们拆开看：
        
        # A. 处理 ID (文件名)
        # [语法]: i.split('.')[0]
        # [作用]: 把 '名字.jpg' 以 '.' 为界切开，取第 0 部分 ('001')。
        # 为什么要切？因为 Kaggle 的提交格式要求 id 不能带 .jpg 后缀。
        # split完是一个列表
        image_id = i.split('.')[0]
        
        # B. 处理概率数值 (List Comprehension)
        # [语法]: [str(num) for num in output]
        # [作用]: output 本来是一堆浮点数 [0.01, 0.99]。
        # join 方法只接受字符串列表，不接受数字。
        # 所以必须用列表推导式，把每个 float 强转成 string。
        str_probs = [str(num) for num in output]
        
        # C. 拼接成 CSV 格式
        # [语法]: ','.join(...)
        # [作用]: 把上面的字符串列表变成 "0.01,0.99"。
        csv_probs = ','.join(str_probs)
        
        # D. 最终写入
        # [作用]: 拼好 ID + 逗号 + 概率串 + 换行符。
        # 结果样子: "001,0.01,0.99\n"
        f.write(image_id + ',' + csv_probs + '\n')

# [语法]: print 打印。
# [作用]: 提示用户程序跑完了，可以去文件夹里找 submission.csv 文件并提交到 Kaggle 了。
print("提交文件 submission.csv 生成完毕！")

64
128
192
256
320
384
448
512
576
640
704
768
832
896
960
1024
1088
1152
1216
1280
1344
1408
1472
1536
1600
1664
1728
1792
1856
1920
1984
2048
2112
2176
2240
2304
2368
2432
2496
2560
2624
2688
2752
2816
2880
2944
3008
3072
3136
3200
3264
3328
3392
3456
3520
3584
3648
3712
3776
3840
3904
3968
4032
4096
4160
4224
4288
4352
4416
4480
4544
4608
4672
4736
4800
4864
4928
4992
5056
5120
5184
5248
5312
5376
5440
5504
5568
5632
5696
5760
5824
5888
5952
6016
6080
6144
6208
6272
6336
6400
6464
6528
6592
6656
6720
6784
6848
6912
6976
7040
7104
7168
7232
7296
7360
7424
7488
7552
7616
7680
7744
7808
7872
7936
8000
8064
8128
8192
8256
8320
8384
8448
8512
8576
8640
8704
8768
8832
8896
8960
9024
9088
9152
9216
9280
9344
9408
9472
9536
9600
9664
9728
9792
9856
9920
9984
10048
10112
10176
10240
10304
10357
提交文件 submission.csv 生成完毕！


In [20]:
a = 'sdhsihd/shdskhd/sdhkahjd'
print(a.split('/'))

['sdhsihd', 'shdskhd', 'sdhkahjd']
